<a href="https://colab.research.google.com/github/ranjanchoubey/ml2/blob/main/assignment-1/Q1-Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- ![image.png](attachment:image.png) -->

![heading](https://raw.githubusercontent.com/ranjanchoubey/ml2/main/assignment-1/assets/heading.png)

![Problem 1](https://raw.githubusercontent.com/ranjanchoubey/ml2/main/assignment-1/assets/q1.1.png)


**Ans : -** **Denoising Autoencoder** (DAE) is a type of neural network that helps to find useful patterns in data.
- It is often used to reduce the number of features or to learn important features.

-  The key idea behind a Denoising Autoencoder is to train the network to reconstruct the original input from a corrupted version of it.

- This process forces the network to learn robust features that can capture the underlying structure of the data, even in the presence of noise.


**Pseudocode for Denoising Autoencoder Training Loop**

---

- **Step 1.** Initialize the autoencoder model $f_{\theta}$
- **Step 2.** Define the loss function $\mathcal{L}$ (e.g., Mean Squared Error)
- **Step 3.** Define the optimizer (e.g., Adam)
- **Step 4.** Set the number of epochs $N$ and batch size $B$
- **Step 5.** For each epoch $n$ from 1 to $N$:
    - For each batch of data $\{x_i\}_{i=1}^{B}$:
        - **i.** Add noise to the input data to create a corrupted version $\tilde{x} = x + \text{noise}$  
        - **ii.** Forward pass: Pass the corrupted data through the autoencoder to get the reconstructed output $\hat{x} = f_{\theta}(\tilde{x})$  
        - **iii.** Compute the loss $\mathcal{L}(\hat{x}, x)$ between the reconstructed output and the original input  
        - **iv.** Backward pass: Compute the gradients of the loss with respect to the model parameters $\nabla_{\theta} \mathcal{L}$  
        - **v.** Update the model parameters using the optimizer $\theta \leftarrow \theta - \eta \nabla_{\theta} \mathcal{L}$       
- **Step 6.** Evaluate the model on a validation set (optional)
- **Step 7.** Save the trained model (optional)

---


![q1.2.png](https://github.com/ranjanchoubey/ml2/blob/main/assignment-1/assets/q1.2.png?raw=1)

In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:


# Step 1: Data Preparation

# Define transformations for the dataset
tensor_transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load the MNIST dataset
train_set = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=tensor_transform)
test_set = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=tensor_transform)

# Create DataLoader for training and testing
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=32, shuffle=True)

# Step 2: Define the Denoising Autoencoder Model

class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        # Define the encoder
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        # Define the decoder
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28*28),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Step 3: Define the Loss Function and Optimizer

model = DenoisingAutoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 4: Training the Denoising Autoencoder

patch_size = 8
num_epochs = 20

for epoch in range(num_epochs):
    for data in train_loader:
        img, _ = data

        # Create a copy of the original images
        noisy_img = img.clone()

        # Add noise to a randomly selected 8x8 patch
        for i in range(noisy_img.size(0)):  # Iterate through each image in the batch
            x_start = torch.randint(0, 28-patch_size, (1,)).item()
            y_start = torch.randint(0, 28-patch_size, (1,)).item()
            noisy_img[i, 0, x_start:x_start+patch_size, y_start:y_start+patch_size] += 0.5 * torch.randn(patch_size, patch_size)

        noisy_img = torch.clamp(noisy_img, 0., 1.)

        # Forward pass: compute the output and loss
        output = model(noisy_img.view(-1, 28*28))
        loss = criterion(output, img.view(-1, 28*28))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Step 5: Visualize the results
    with torch.no_grad():
        output = output.view(-1, 1, 28, 28)
        img = img.view(-1, 1, 28, 28)
        noisy_img = noisy_img.view(-1, 1, 28, 28)

        # Plotting
        fig, axes = plt.subplots(1, 3, figsize=(9, 3))
        axes[0].imshow(img[0].cpu().squeeze(), cmap='gray')
        axes[0].set_title('Original Image')
        axes[0].axis('off')

        axes[1].imshow(noisy_img[0].cpu().squeeze(), cmap='gray')
        axes[1].set_title('Noisy Image (Patch)')
        axes[1].axis('off')

        axes[2].imshow(output[0].cpu().squeeze(), cmap='gray')
        axes[2].set_title('Denoised Image')
        axes[2].axis('off')

        plt.show()

# Step 6: Evaluate the Model
# Optionally, you can evaluate the model on the test set using the same approach
